## ROI POOLING
- Implement a ROI Pooling operator. Your code will be given the following variables:
  - input, a mini-batch of feature maps (a torch.Tensor with shape (n, C, H, W) and dtype torch.float32)
  - boxes, a list of bounding box coordinates on which you need to perform the ROI Pooling. boxes will be a list of (L,4) torch.Tensor with dtype torch.float32, where boxes[i] will refer to the i-th element of the batch, and contain L coordinates in the format (y1, x1, y2, x2)
  - a tuple of integers output_size, containing the number of cells over which pooling is performed, in the format (heigth, width)
- The code should produce an output torch.Tensor out with dtype torch.float32 and shape (n, L, C, output_size[0], output_size[1]).

In [ ]:
import random
import torch

n = random.randint(1, 3)
C = random.randint(10, 20)
H = random.randint(5, 10)
W = random.randint(5, 10)
oH = random.randint(2, 4)
oW = random.randint(2, 4)
L = random.randint(2, 6)
input = torch.rand(n, C, H, W)
boxes = [torch.zeros(L, 4) for _ in range(n)]
for i in range(n):
  boxes[i][:, 0] = torch.rand(L) * (H-oH)       # y
  boxes[i][:, 1] = torch.rand(L) * (W-oW)       # x
  boxes[i][:, 2] = oH + torch.rand(L) * (H-oH)  # w
  boxes[i][:, 3] = oW + torch.rand(L) * (W-oW)  # h

  boxes[i][:,2:] += boxes[i][:,:2]
  boxes[i][:,2] = torch.clamp(boxes[i][:,2], max=H-1)
  boxes[i][:,3] = torch.clamp(boxes[i][:,3], max=W-1)
output_size = (oH, oW)



In [ ]:
import torch.nn.functional as F

def roi_pooling(input, boxes, output_size):
    n, C, H, W = input.shape
    oH, oW = output_size
    L = boxes[0].size(0)

    output = torch.zeros(n, L, C, oH, oW)

    for i in range(n):
        for j in range(L):
            box = boxes[i][j]
            y1, x1, y2, x2 = box

            # Ensure the box coordinates are within bounds
            y1, x1, y2, x2 = int(y1), int(x1), int(y2), int(x2)
            y1, y2 = max(0, y1), min(H, y2)
            x1, x2 = max(0, x1), min(W, x2)

            # Perform ROI Pooling
            roi_features = F.adaptive_max_pool2d(input[i, :, y1:y2, x1:x2], output_size)
            output[i, j, :, :, :] = roi_features

    return output

# Example usage with the provided variables
output = roi_pooling(input, boxes, output_size)
print(output.shape)